# Simple N(utrient)-P(hytoplankton)-Z(ooplankton) Model
Using an explicit time-differencing scheme to solve a simple NPZ model.

In [210]:
from ipywidgets import interact
import ipywidgets as widgets
import numpy as np
from bokeh.io import output_file, show, output_notebook, push_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

Loading BokehJS ...

In [211]:
# + + + Parameters + + + 
Vm        = 1    # Maximum growth rate (per day)
Kn        = 1    # Half-saturation constant for nitrogen uptake (umolN per l)
Rm        = 1    # Maximum grazing rate (per day)
g         = 0.2  # Zooplankton death rate (per day)
lambda_Z  = 0.2  # Grazing constant (umolN per l)
epsilon   = 0.1  # Phyto death rate (per day)
gamma_Z   = 0.7  # Dimensionless proportion of assimilated nitrogen by Zooplankton
f         = 0.25 # Light intensity (assumed constant)
dt        = 1    # Timestep of 1 day

In [212]:
# + + + Run model with your own initial conditions for base plot.
Num = 100
N_0 = 4
P_0 = 2.5
Z_0 = 0.5

x = np.arange(1, Num + 1, 1)

# Initialize arrays for time series.
N = np.empty(Num)
P = np.empty(Num)
Z = np.empty(Num)

# Fill with initialized values
N[0] = N_0
P[0] = P_0
Z[0] = Z_0

for idx in np.arange(1, Num, 1):
    t = idx - 1
    
    # Common terms.
    gamma_N   = N[t] / (Kn + N[t])
    zoo_graze = Rm * (1 - np.exp(-lambda_Z*P[t])) * Z[t]

    # Variables
    N[idx] = dt * (-Vm*gamma_N*f*P[t] + (1-gamma_Z)*zoo_graze + epsilon*P[t] + g*Z[t]) + N[t]
    P[idx] = P[t]*(1 - epsilon*dt + Vm*gamma_N*f*dt) - (zoo_graze * dt);
    Z[idx] = dt * (gamma_Z*zoo_graze - g*Z[t]) + Z[t]   

In [213]:
# + + + Initial plot that is shown +++++
source = ColumnDataSource(data=dict(x=x, y0=N, y1=P, y2=Z))

# First plot
s1 = figure(width=600, plot_height=200, title="Nutrients",
           x_range=(1,100), y_range=(0,7))
p1 = s1.circle('x', 'y0', source=source, size=2, color="navy", alpha=0.5)

# Second plot
s2 = figure(width=600, height=200, x_range=s1.x_range, y_range=s1.y_range, title="Phyto")
p2 = s2.circle('x', 'y1', source=source, size=2, color="firebrick", alpha=0.5)

# Third plot
s3 = figure(width=600, height=200, x_range=s1.x_range, y_range=s1.y_range, title="Zoo")
p3 = s3.circle('x', 'y2', source=source, size=2, color="olive", alpha=0.5)

p = gridplot([s1], 
             [s2], 
             [s3], toolbar_location=None)

In [214]:
# + + + Run Model + + + 
# This updates the plot live. The parameters set here set the defaults on the sliders.
def update(Num=100, N_0=4, P_0=2.5, Z_0=0.5):
    x = np.arange(1, Num + 1, 1) # Days of simulation

    # Initialize arrays for time series.
    N = np.empty(Num)
    P = np.empty(Num)
    Z = np.empty(Num)

    # Fill with initialized values
    N[0] = N_0
    P[0] = P_0
    Z[0] = Z_0
    
    # Run model.
    for idx in np.arange(1, Num, 1):
        t = idx - 1
    
        # Common terms.
        gamma_N   = N[t] / (Kn + N[t])
        zoo_graze = Rm * (1 - np.exp(-lambda_Z*P[t])) * Z[t]

        # Variables
        N[idx] = dt * (-Vm*gamma_N*f*P[t] + (1-gamma_Z)*zoo_graze + epsilon*P[t] + g*Z[t]) + N[t]
        P[idx] = P[t]*(1 - epsilon*dt + Vm*gamma_N*f*dt) - (zoo_graze * dt);
        Z[idx] = dt * (gamma_Z*zoo_graze - g*Z[t]) + Z[t]
        
    # Push to plot
    p1.data_source.data['x']  = x
    p1.data_source.data['y0'] = N
    
    p2.data_source.data['y1'] = P
    p2.data_source.data['x']  = x
    
    p3.data_source.data['y2'] = Z
    p3.data_source.data['x']  = x
    
    s1.y_range.start = 0
    s1.y_range.end   = np.ceil(np.max(N)) + 1
    
    push_notebook()

In [215]:
# show the results
show(p, notebook_handle=True)

In [216]:
interact(update, Num=(1, 1000, 10), N_0=(0, 10, 0.1), P_0=(0, 10, 0.1), Z_0=(0, 10, 0.1))

<function __main__.update>